In [1]:
import pandas as pd
from pymongo import MongoClient
import re
import numpy as np

In [2]:
client = MongoClient("mongodb://localhost:27017/")

In [3]:
db = client.companies

In [4]:
companies = db.companies.find()

In [5]:
data_companies = pd.DataFrame(companies)

VISUALIZAMOS EL DATASET

In [28]:
data_companies['category_code'].value_counts()

web                 3787
software            2736
games_video         1083
mobile              1018
other                986
advertising          928
enterprise           742
ecommerce            688
consulting           637
network_hosting      626
public_relations     533
search               394
biotech              373
hardware             368
cleantech            305
semiconductor        167
security             156
analytics             66
social                49
finance               49
news                  48
education             36
music                 33
messaging             30
travel                25
legal                 25
medical               25
photo_video           23
health                23
manufacturing         19
sports                13
fashion               10
real_estate           10
automotive             9
hospitality            8
transportation         7
nanotech               5
design                 4
nonprofit              4
local                  1


SACAMOS QUERY

In [6]:
gaming = db.companies.find({"category_code":'games_video', "founded_year": {"$gt":1990}, "number_of_employees": {"$gt": 25}},
 {"name":1, "founded_year":1, "offices":1, "category_code":1, "total_money_raised":1, "number_of_employees":1,'description':1})

In [7]:
office_companies = pd.DataFrame(gaming)
office_companies.head()

,_id,name,category_code,number_of_employees,founded_year,description,total_money_raised,offices
0,52cdef7c4bab8bd675297dab,Kyte,games_video,40,2006,Online & Mobile Video Platform,$23.4M,"[{'description': None, 'address1': '442 Post S..."
1,52cdef7c4bab8bd675297dbe,Stickam,games_video,35,2006,,$0,"[{'description': '', 'address1': '1975 pads rd..."
2,52cdef7c4bab8bd675297dc1,Ustream,games_video,250,2007,Live Social Video Community,$60.1M,"[{'description': 'San Francisco Office', 'addr..."
3,52cdef7c4bab8bd675297dc0,Livestream,games_video,120,2007,Live streaming platform and website,$14.7M,"[{'description': 'Livestream HQ', 'address1': ..."
4,52cdef7c4bab8bd675297df0,RockYou,games_video,106,2005,Social Gaming Company,$136M,"[{'description': '', 'address1': '585 Broadway..."


IMPORTAMOS FUNCIONES PARA SACAR LONGITUD Y LATITUD

In [11]:
import coordenadas

    
office_companies['longitud'] = office_companies.apply(lambda x: get_lon(x), axis=1)
office_companies.head()

,_id,name,category_code,number_of_employees,founded_year,description,total_money_raised,offices,longitud
0,52cdef7c4bab8bd675297dab,Kyte,games_video,40,2006,Online & Mobile Video Platform,$23.4M,"[{'description': None, 'address1': '442 Post S...",-122.409173
1,52cdef7c4bab8bd675297dbe,Stickam,games_video,35,2006,,$0,"[{'description': '', 'address1': '1975 pads rd...",-118.254558
2,52cdef7c4bab8bd675297dc1,Ustream,games_video,250,2007,Live Social Video Community,$60.1M,"[{'description': 'San Francisco Office', 'addr...",-122.079480
3,52cdef7c4bab8bd675297dc0,Livestream,games_video,120,2007,Live streaming platform and website,$14.7M,"[{'description': 'Livestream HQ', 'address1': ...",-73.995625
4,52cdef7c4bab8bd675297df0,RockYou,games_video,106,2005,Social Gaming Company,$136M,"[{'description': '', 'address1': '585 Broadway...",-122.206893


In [16]:
import coordenadas


In [17]:
office_companies['latitude'] = office_companies.apply(lambda x: get_lat(x), axis=1)
office_companies

,_id,name,category_code,number_of_employees,founded_year,description,total_money_raised,offices,longitud,latitude
0,52cdef7c4bab8bd675297dab,Kyte,games_video,40,2006,Online & Mobile Video Platform,$23.4M,"[{'description': None, 'address1': '442 Post S...",-122.409173,37.788482
1,52cdef7c4bab8bd675297dbe,Stickam,games_video,35,2006,,$0,"[{'description': '', 'address1': '1975 pads rd...",-118.254558,34.051409
2,52cdef7c4bab8bd675297dc1,Ustream,games_video,250,2007,Live Social Video Community,$60.1M,"[{'description': 'San Francisco Office', 'addr...",-122.079480,37.392936
3,52cdef7c4bab8bd675297dc0,Livestream,games_video,120,2007,Live streaming platform and website,$14.7M,"[{'description': 'Livestream HQ', 'address1': ...",-73.995625,40.726155
4,52cdef7c4bab8bd675297df0,RockYou,games_video,106,2005,Social Gaming Company,$136M,"[{'description': '', 'address1': '585 Broadway...",-122.206893,37.484619
5,52cdef7c4bab8bd675297e11,iLike,games_video,28,2002,social music discovery site,$16.5M,"[{'description': None, 'address1': '1605 Boyls...",-122.323408,47.615313
6,52cdef7c4bab8bd675297e44,hi5,games_video,100,2003,,$52M,"[{'description': '', 'address1': '55 Second St...",-122.400558,37.788668
7,52cdef7c4bab8bd675297e55,Curse,games_video,58,2006,Online games media company,$12M,"[{'description': 'San Francisco', 'address1': ...",-122.399972,37.787092
8,52cdef7c4bab8bd675297eff,Zango,games_video,225,1999,,$40M,"[{'description': None, 'address1': '3600 136th...",-122.155668,47.579208
9,52cdef7c4bab8bd675297f71,OMGPOP,games_video,50,2006,Free online multiplayer game,$16.6M,"[{'description': '', 'address1': 'SoHo', 'addr...",-74.001704,40.723384


In [18]:
office_companies.dropna(subset=['latitude'], inplace=True)
office_companies.dropna(subset=['longitud'], inplace=True)

In [19]:
office_companies.head()

,_id,name,category_code,number_of_employees,founded_year,description,total_money_raised,offices,longitud,latitude
0,52cdef7c4bab8bd675297dab,Kyte,games_video,40,2006,Online & Mobile Video Platform,$23.4M,"[{'description': None, 'address1': '442 Post S...",-122.409173,37.788482
1,52cdef7c4bab8bd675297dbe,Stickam,games_video,35,2006,,$0,"[{'description': '', 'address1': '1975 pads rd...",-118.254558,34.051409
2,52cdef7c4bab8bd675297dc1,Ustream,games_video,250,2007,Live Social Video Community,$60.1M,"[{'description': 'San Francisco Office', 'addr...",-122.079480,37.392936
3,52cdef7c4bab8bd675297dc0,Livestream,games_video,120,2007,Live streaming platform and website,$14.7M,"[{'description': 'Livestream HQ', 'address1': ...",-73.995625,40.726155
4,52cdef7c4bab8bd675297df0,RockYou,games_video,106,2005,Social Gaming Company,$136M,"[{'description': '', 'address1': '585 Broadway...",-122.206893,37.484619


IMPORTAMOS FUNCIÓN GEO PARA SACAR EL GEO INDICE

In [24]:
import geo

In [25]:
df = pd.DataFrame(office_companies)
df['position'] = df.apply(lambda x: createGeoJson(x["longitud"],x["latitude"]), axis=1)
display(df.head())
df.shape

,_id,name,category_code,number_of_employees,founded_year,description,total_money_raised,offices,longitud,latitude,position
0,52cdef7c4bab8bd675297dab,Kyte,games_video,40,2006,Online & Mobile Video Platform,$23.4M,"[{'description': None, 'address1': '442 Post S...",-122.409173,37.788482,"{'type': 'Point', 'coordinates': [-122.409173,..."
1,52cdef7c4bab8bd675297dbe,Stickam,games_video,35,2006,,$0,"[{'description': '', 'address1': '1975 pads rd...",-118.254558,34.051409,"{'type': 'Point', 'coordinates': [-118.254558,..."
2,52cdef7c4bab8bd675297dc1,Ustream,games_video,250,2007,Live Social Video Community,$60.1M,"[{'description': 'San Francisco Office', 'addr...",-122.079480,37.392936,"{'type': 'Point', 'coordinates': [-122.07948, ..."
3,52cdef7c4bab8bd675297dc0,Livestream,games_video,120,2007,Live streaming platform and website,$14.7M,"[{'description': 'Livestream HQ', 'address1': ...",-73.995625,40.726155,"{'type': 'Point', 'coordinates': [-73.995625, ..."
4,52cdef7c4bab8bd675297df0,RockYou,games_video,106,2005,Social Gaming Company,$136M,"[{'description': '', 'address1': '585 Broadway...",-122.206893,37.484619,"{'type': 'Point', 'coordinates': [-122.206893,..."


(71, 11)

In [26]:
df = df.drop('_id', axis=1)

In [27]:
df

,name,category_code,number_of_employees,founded_year,description,total_money_raised,offices,longitud,latitude,position
0,Kyte,games_video,40,2006,Online & Mobile Video Platform,$23.4M,"[{'description': None, 'address1': '442 Post S...",-122.409173,37.788482,"{'type': 'Point', 'coordinates': [-122.409173,..."
1,Stickam,games_video,35,2006,,$0,"[{'description': '', 'address1': '1975 pads rd...",-118.254558,34.051409,"{'type': 'Point', 'coordinates': [-118.254558,..."
2,Ustream,games_video,250,2007,Live Social Video Community,$60.1M,"[{'description': 'San Francisco Office', 'addr...",-122.079480,37.392936,"{'type': 'Point', 'coordinates': [-122.07948, ..."
3,Livestream,games_video,120,2007,Live streaming platform and website,$14.7M,"[{'description': 'Livestream HQ', 'address1': ...",-73.995625,40.726155,"{'type': 'Point', 'coordinates': [-73.995625, ..."
4,RockYou,games_video,106,2005,Social Gaming Company,$136M,"[{'description': '', 'address1': '585 Broadway...",-122.206893,37.484619,"{'type': 'Point', 'coordinates': [-122.206893,..."
5,iLike,games_video,28,2002,social music discovery site,$16.5M,"[{'description': None, 'address1': '1605 Boyls...",-122.323408,47.615313,"{'type': 'Point', 'coordinates': [-122.323408,..."
6,hi5,games_video,100,2003,,$52M,"[{'description': '', 'address1': '55 Second St...",-122.400558,37.788668,"{'type': 'Point', 'coordinates': [-122.400558,..."
7,Curse,games_video,58,2006,Online games media company,$12M,"[{'description': 'San Francisco', 'address1': ...",-122.399972,37.787092,"{'type': 'Point', 'coordinates': [-122.399972,..."
8,Zango,games_video,225,1999,,$40M,"[{'description': None, 'address1': '3600 136th...",-122.155668,47.579208,"{'type': 'Point', 'coordinates': [-122.155668,..."
9,OMGPOP,games_video,50,2006,Free online multiplayer game,$16.6M,"[{'description': '', 'address1': 'SoHo', 'addr...",-74.001704,40.723384,"{'type': 'Point', 'coordinates': [-74.001704, ..."


EXPORTAMOS DATASET MODIFICADO A JSON

In [ ]:
df.to_json(r'./offices2.json', orient="records")

In [ ]:
#mongoimport --db companies --collection limpiado --jsonArray offices.json ---> para importar el archivo json a Mongo
